In [12]:
from pybaseball import statcast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import plotly.express as px

In [2]:
#import all pitches from 2021 into dataframe
statcast_data = statcast('2020-12-01', '2021-12-01')

This is a large query, it may take a moment to complete


/opt/anaconda3/envs/minimal_ds/lib/python3.10/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


100%|██████████| 246/246 [01:08<00:00,  3.59it/s]


In [8]:
statcast_data.columns #view all columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [11]:
#determine relevant columns --> these columns were chosen by intuition. I am only interested in using statcast data to predict the pitch
df_relevant = statcast_data[['pitch_type','release_speed','p_throws','pfx_x','pfx_z','plate_x','plate_z','vx0','vy0','vz0','ax','ay','az','release_spin_rate','pitch_name','spin_axis']]

In [13]:
df_relevant.isna().sum() #see how many null values are in the dataset

pitch_type            365
release_speed         402
p_throws                0
pfx_x                 368
pfx_z                 368
plate_x               368
plate_z               368
vx0                   368
vy0                   368
vz0                   368
ax                    368
ay                    368
az                    368
release_spin_rate    3028
pitch_name            365
spin_axis            2992
dtype: int64

Most of the columns has 368 missing values. This leads me to believe it is the same rows for each of the columns. Since there are over 700,000 rows in the dataset, it is probably best to remove these rows. The same may be said for the spin rate and spin axis. 

In [15]:
df_relevant.nunique()

pitch_type               13
release_speed           583
p_throws                  2
pfx_x                   461
pfx_z                   430
plate_x                 795
plate_z                 942
vx0                  720268
vy0                  720269
vz0                  720268
ax                   720269
ay                   720269
az                   720269
release_spin_rate      3105
pitch_name               12
spin_axis               361
dtype: int64

I noticed that pitch_type and pitch_name have different numbers of unique values, although they are both referring to the same thing. I investigated below

In [24]:
df_relevant['pitch_type'].unique()

array(['FF', 'SL', 'CU', 'CH', 'FS', 'SI', 'KC', 'FC', 'CS', nan, 'FA',
       'KN', 'EP', 'SC'], dtype=object)

In [23]:
df_relevant['pitch_name'].unique()

array(['4-Seam Fastball', 'Slider', 'Curveball', 'Changeup',
       'Split-Finger', 'Sinker', 'Knuckle Curve', 'Cutter', nan,
       'Fastball', 'Knuckleball', 'Eephus', 'Screwball'], dtype=object)

The pitch_type column as a value 'CS' that does not correspond to anything from the pitch_name column

In [28]:
df_relevant[df_relevant['pitch_type'] == 'CS'].head(5)

,pitch_type,release_speed,p_throws,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,pitch_name,spin_axis
14,CS,67.8,R,1.43,-1.3,1.69,1.0,3.100946,-98.54461,0.446421,8.514053,19.472324,-40.849224,2622,Curveball,39
60,CS,69.8,L,-2.15,-0.24,-0.77,2.49,-2.556083,-101.365132,4.623964,-14.039311,20.242167,-34.957753,2636,Curveball,274
81,CS,67.9,L,-2.33,0.06,-0.07,3.23,-1.146932,-98.499702,5.811309,-14.679986,19.235388,-33.190684,2744,Curveball,273
150,CS,67.7,L,-2.31,0.16,-0.6,3.03,-1.84581,-98.216616,5.715581,-14.255159,20.236141,-32.618932,2697,Curveball,272
168,CS,66.6,L,-2.28,0.25,1.04,3.01,1.035509,-96.632806,5.622793,-14.220294,18.894316,-31.992749,2649,Curveball,270


It looks like 'CS' is being classified as 'curveball'

In [29]:
df_relevant[df_relevant['pitch_type'] == 'CU'].head(5)

,pitch_type,release_speed,p_throws,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,pitch_name,spin_axis
266,CU,77.2,L,-0.79,-1.09,1.06,4.8,0.288752,-112.269839,4.402666,-6.664866,23.089543,-42.468284,1938,Curveball,325
204,CU,84.4,R,0.62,-0.03,0.14,1.53,4.887751,-122.936949,-2.193857,5.357669,23.999432,-32.271503,2562,Curveball,76
217,CU,82.0,R,1.35,-0.56,1.19,0.39,5.812482,-119.201895,-3.287973,11.467446,26.102005,-36.951864,2778,Curveball,67
227,CU,83.5,R,1.43,-0.52,0.44,1.51,3.944388,-121.494864,-1.066441,13.157535,26.76832,-37.267542,2810,Curveball,68
240,CU,82.5,R,0.82,-0.25,-0.08,2.42,4.278948,-120.133698,0.524597,7.147985,22.252259,-34.942785,2559,Curveball,73


Both 'CU' and 'CS' refer to curveball, although 'CU is much more prevelant

In [30]:
#drop pitch_type column due to label confusion
df_drop_type = df_relevant.drop(columns='pitch_type')